In [140]:
import time

import numpy as np
import pandas as pd
import yfinance as yf

In [141]:
class Account:
    def __init__(self, test_name, starting_balance):
        self.name = test_name
        self.assets = {}  # 'ticker': {'share_count':#, 'share_value':#, 'cost_basis':#}
        self.balance = starting_balance
        self.account_value = starting_balance
        
    def update_account_value(self):
        try:
            self.account_value = self.balance
            for key in self.assets.keys():
                self.account_value += self.assets[key]['share_value']
        except KeyError:
            print('Attempted to access share_count or share_price when they do not exist')
            
    def buy(self, ticker, shares, price):
        try:
            sale_price = shares*price
            self.balance -= sale_price

            if ticker not in self.assets.keys():
                self.assets[ticker] = {'share_count':0, 'cost_basis':0}

            prior_cost_basis = self.assets[ticker]['cost_basis']
            prior_share_count = self.assets[ticker]['share_count']
            self.assets[ticker]['cost_basis'] = ((prior_cost_basis*prior_share_count)+sale_price)/(prior_share_count+shares)

            self.assets[ticker]['share_count'] += shares
            self.assets[ticker]['share_value'] = self.assets[ticker]['share_count']*price
        except ZeroDivisionError:
            print('Tried to buy 0 shares when there are no shares existing in account')

    
    def sell(self, ticker, shares, price):
        try:
            sale_price = shares*price
            self.balance += sale_price

            self.assets[ticker]['share_count'] -= shares
            self.assets[ticker]['share_value'] = self.assets[ticker]['share_count']*price
        except KeyError:
            print('Tried to sell {}, of which there are no existing shares'.format(ticker))
            
    def hold(self, ticker, price):
        try:
            self.assets[ticker]['share_value'] = self.assets[ticker]['share_count']*price
        except KeyError:
            print('Tried to hold onto an asset that does not exist')

In [154]:
def rename_with_symbols(df, symbol):
    for col in df.columns:
        if '_' not in col:
            df.rename(columns={col: "{}_{}".format(symbol,col)}, inplace=True)
    return(None)

In [155]:
ticklist = ['^GSPC','UPRO']
total_df = yf.Ticker(ticklist[0]).history(period="max")

rename_with_symbols(total_df, 'GSPC')

for i,ticker in enumerate(ticklist[1:]):
    ran = np.random.uniform(low=-1, high=1)*2
    time.sleep(5+ran)

    try:
        new_df = yf.Ticker(ticker).history(period="max")
        rename_with_symbols(new_df, ticker)
        total_df = total_df.join(new_df, how='outer')
    except:
        print("failed {}".format(ticker))
        next
        
    if i%10 == 0:
        print("{}% Gathered".format(i/len(ticklist)))
    
total_df = total_df[[col for col in total_df.columns if 'Close' in col]]
df = total_df.copy(deep=True)
total_df.tail()

0.0% Gathered


,GSPC_Close,UPRO_Close
Date,,
2020-04-30,2912.43,37.90
2020-05-01,2830.71,34.86
2020-05-04,2842.74,35.23
2020-05-05,2868.44,36.17
2020-05-06,2848.42,35.45


In [156]:
start = '2020-04-29'
df = df[df.index >= start]
df

,GSPC_Close,UPRO_Close
Date,,
2020-04-29,2939.51,38.99
2020-04-30,2912.43,37.90
2020-05-01,2830.71,34.86
2020-05-04,2842.74,35.23
2020-05-05,2868.44,36.17
2020-05-06,2848.42,35.45


In [157]:
account_tracker = []
test = Account('test',1000000)

for i,d in enumerate(df.index):
    print(d)
    
    if i == 4:
        price = df.loc[d,'GSPC_Close']
        test.hold('GSPC_Close',price)
        test.hold('UPRO_Close',uprice)
        print('Held GSPC at {}'.format(price))
    elif i in [0,3]:
        price = df.loc[d,'GSPC_Close']
        shares = np.floor(test.balance/2/price)
        test.buy('GSPC_Close',shares,price)
        test.hold('UPRO_Close',price)
        print('Bought {} shares of GSPC at {}'.format(shares,price))
    elif i == 1:
        price = df.loc[d,'UPRO_Close']
        shares = np.floor(test.balance/3/price)
        test.buy('UPRO_Close',shares,price)
        print('Bought {} shares of UPRO at {}'.format(shares,price))
    elif i == 2:
        price = df.loc[d,'UPRO_Close']
        try:
            shares = np.floor(test.assets['UPRO']['share_count']/10)
            test.sell('UPRO_Close',shares,price)
            print('Sold {} shares of UPRO at {}'.format(shares,price))
        except KeyError:
            print('Tried to sell {}, of which there are no existing shares'.format('UPRO'))
    else:
        price = df.loc[d,'GSPC_Close']
        try:
            shares = np.floor(test.assets['GSPC']['share_count']/10)
            test.sell('GSPC_Close',shares,price)
            print('Sold {} shares of GSPC at {}'.format(shares,price))
        except KeyError:
            print('Tried to sell {}, of which there are no existing shares'.format('GSPC'))
        
    print(test.assets)
    test.update_account_value()
    account_tracker.append(test.account_value)
    print("")
        
test.assets

2020-04-29 00:00:00
Bought 170.0 shares of GSPC at 2939.51
{'GSPC_Close': {'share_count': 170.0, 'cost_basis': 2939.51, 'share_value': 499716.7}}

2020-04-30 00:00:00
Bought 4400.0 shares of UPRO at 37.9
{'GSPC_Close': {'share_count': 170.0, 'cost_basis': 2939.51, 'share_value': 499716.7}, 'UPRO_Close': {'share_count': 4400.0, 'cost_basis': 37.9, 'share_value': 166760.0}}

2020-05-01 00:00:00
Tried to sell UPRO, of which there are no existing shares
{'GSPC_Close': {'share_count': 170.0, 'cost_basis': 2939.51, 'share_value': 499716.7}, 'UPRO_Close': {'share_count': 4400.0, 'cost_basis': 37.9, 'share_value': 166760.0}}

2020-05-04 00:00:00
Bought 58.0 shares of GSPC at 2842.74
{'GSPC_Close': {'share_count': 228.0, 'cost_basis': 2914.8930701754384, 'share_value': 648144.72}, 'UPRO_Close': {'share_count': 4400.0, 'cost_basis': 37.9, 'share_value': 166760.0}}

2020-05-05 00:00:00
Held GSPC at 2868.44
{'GSPC_Close': {'share_count': 228.0, 'cost_basis': 2914.8930701754384, 'share_value': 6540

{'GSPC_Close': {'share_count': 228.0,
  'cost_basis': 2914.8930701754384,
  'share_value': 654004.3200000001},
 'UPRO_Close': {'share_count': 4400.0,
  'cost_basis': 37.9,
  'share_value': 166760.0}}

In [158]:
account_tracker

[1000000.0,
 1000000.0,
 1000000.0,
 983549.1,
 989408.7000000001,
 989408.7000000001]